# Tratamento dos Dados

In [1]:
import pandas as pd
from unidecode import unidecode

In [2]:
# url = 'https://bi.static.es.gov.br/covid19/MICRODADOS.csv'
url = 'MICRODADOS.zip'
df = pd.read_csv(url, sep = ';', encoding = 'cp1252')
df.head()

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido
0,2021-04-22,2021-03-19,2021-04-19,2021-04-22,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não Informado,Ignorado,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado
1,2021-04-22,2020-06-26,2021-04-16,NaN,2021-04-22,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Sim,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
2,2021-04-22,2021-04-13,2021-04-07,2021-04-22,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não Informado,Sim,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado
3,2021-04-22,2021-04-15,2021-04-19,2021-04-22,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado
4,2021-04-22,2020-12-20,2021-04-18,NaN,2021-04-22,NaN,NaN,2020-12-26,NaN,Descartados,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno


In [3]:
df.describe().T

,count,unique,top,freq
DataNotificacao,1296195,422,2021-03-22,9072
DataCadastro,1296195,392,2021-03-24,8550
DataDiagnostico,1296195,462,2021-03-15,7840
DataColeta_RT_PCR,653231,512,2021-03-22,5104
DataColetaTesteRapido,397005,468,2020-10-21,3977
DataColetaSorologia,77685,399,2020-07-20,963
DataColetaSorologiaIGG,120592,416,2020-07-28,2728
DataEncerramento,1095360,468,2021-04-05,8083
DataObito,11720,387,2021-04-05,86
Classificacao,1296195,3,Descartados,628430


In [4]:
print('Classificacão: {}'.format(df['Classificacao'].unique()))
print('Evolução: {}'.format(df['Evolucao'].unique()))

Classificacão: ['Suspeito' 'Descartados' 'Confirmados']
Evolução: ['-' 'Ignorado' 'Cura' 'Óbito pelo COVID-19' 'Óbito por outras causas']


In [5]:
total_casos = df.query('Classificacao == "Confirmados"').shape[0]
total_obitos = df.query('Evolucao == "Óbito pelo COVID-19"').shape[0]
total_curas = df.query('Evolucao == "Cura"').shape[0]

print('TOTAIS')
print('=================')
print('  - CASOS: {}'.format(total_casos))
print('  - ÓBITOS: {} ({}%)'.format(total_obitos, round((total_obitos * 100) / total_casos, 1)))
print('  - CURA: {} ({}%)'.format(total_curas, round((total_curas * 100) / total_casos, 1)))

TOTAIS
  - CASOS: 423974
  - ÓBITOS: 9012 (2.1%)
  - CURA: 399226 (94.2%)


In [6]:
df['DataNotificacao'] = pd.to_datetime(df['DataNotificacao'])
df.sort_values('DataNotificacao', inplace=True)
df = df.query('Classificacao == "Confirmados"').reset_index(drop=True)

## Padroniza nome de municípios e bairros

In [7]:
df['Municipio'] = df['Municipio'].apply(lambda x: unidecode(str(x)).upper())
df['Bairro'] = df['Bairro'].apply(lambda x: unidecode(str(x)).upper())

## Cálculo de Confirmados, Óbitos e Curados

In [8]:
df['Obitos'] = df['Evolucao'].apply(lambda e: 1 if e == 'Óbito pelo COVID-19' else 0)
df['Confirmados'] = 1
df['Curas'] = df['Evolucao'].apply(lambda e: 1 if e == 'Cura' else 0)

df = df[['Municipio', 'Bairro', 'DataNotificacao', 'Obitos', 'Confirmados', 'Curas']]\
    .groupby(['Municipio', 'Bairro', 'DataNotificacao'])\
    .sum()\
    .reset_index()

In [9]:
grupo_base = ['DataNotificacao', 'Municipio', 'Bairro']

datas = df[['DataNotificacao']].drop_duplicates().reset_index(drop=True)
municipios_bairros = df[['Municipio', 'Bairro']].drop_duplicates().reset_index(drop=True)

datas['key'] = 0
municipios_bairros['key'] = 0

In [10]:
df_counts = pd.merge(datas, municipios_bairros, how='outer')[grupo_base]

df_counts = df_counts.merge(
    df[[
        'Municipio',
        'Bairro',
        'DataNotificacao',
        'Confirmados',
        'Obitos',
        'Curas',
    ]].drop_duplicates(grupo_base, keep='last')\
        .dropna(),
    on=grupo_base,
    how='left'
)

df_counts.fillna(
    {
        'Confirmados': 0,
        'Obitos': 0,
        'Curas': 0,
    },
    inplace=True
)

In [11]:
df_counts.query('Municipio == "VILA VELHA" & DataNotificacao == "2021-04-09"')['Obitos'].sum()

4.0

In [12]:
columns_sum = ['Confirmados', 'Obitos', 'Curas']
df_counts_by_week = df_counts.groupby(['Municipio', 'Bairro', pd.Grouper(key='DataNotificacao', freq='7D')])[columns_sum]\
    .sum()\
    .reset_index()\
    .sort_values('DataNotificacao')

df_counts_by_week['ConfirmadosAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Confirmados']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df_counts_by_week['ObitosAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Obitos']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df_counts_by_week['CurasAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Curas']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df_counts_by_week.query('Municipio == "SERRA" & Bairro == "MORADA DE LARANJEIRAS"').tail(6)

,Municipio,Bairro,DataNotificacao,Confirmados,Obitos,Curas,ConfirmadosAcumulado,ObitosAcumulado,CurasAcumulado
253674,SERRA,MORADA DE LARANJEIRAS,2021-03-13,66.0,0.0,65.0,1762.0,18.0,1740.0
253675,SERRA,MORADA DE LARANJEIRAS,2021-03-20,56.0,0.0,56.0,1818.0,18.0,1796.0
253676,SERRA,MORADA DE LARANJEIRAS,2021-03-27,55.0,1.0,50.0,1873.0,19.0,1846.0
253677,SERRA,MORADA DE LARANJEIRAS,2021-04-03,44.0,0.0,38.0,1917.0,19.0,1884.0
253678,SERRA,MORADA DE LARANJEIRAS,2021-04-10,43.0,1.0,25.0,1960.0,20.0,1909.0
253679,SERRA,MORADA DE LARANJEIRAS,2021-04-17,21.0,2.0,5.0,1981.0,22.0,1914.0


In [13]:
# df_counts_by_week.to_csv('microdados_pre-processed.csv', sep=',', index=False, encoding='UTF-8')